# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
file_path_list[:10]

['/home/workspace/event_data/2018-11-30-events.csv',
 '/home/workspace/event_data/2018-11-23-events.csv',
 '/home/workspace/event_data/2018-11-22-events.csv',
 '/home/workspace/event_data/2018-11-29-events.csv',
 '/home/workspace/event_data/2018-11-11-events.csv',
 '/home/workspace/event_data/2018-11-14-events.csv',
 '/home/workspace/event_data/2018-11-20-events.csv',
 '/home/workspace/event_data/2018-11-15-events.csv',
 '/home/workspace/event_data/2018-11-05-events.csv',
 '/home/workspace/event_data/2018-11-28-events.csv']

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [6]:
full_data_rows_list[1]

['Manowar',
 'Logged In',
 'Jacob',
 'M',
 '0',
 'Klein',
 '247.562',
 'paid',
 'Tampa-St. Petersburg-Clearwater, FL',
 'PUT',
 'NextSong',
 '1.54056E+12',
 '1049',
 'Shell Shock',
 '200',
 '1.54354E+12',
 '73']

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
#Create a Keyspace 
session.execute("""     
                        create keyspace if not exists pro
                        with replication = {'class' : 'SimpleStrategy', 'replication_factor': 1} 
                        
                        """ )

#### Set Keyspace

In [9]:
#Set KEYSPACE to the keyspace specified above
session.set_keyspace('pro')

In [10]:
def execute_query(select):
    try:
        results = session.execute(select)
    except Exception as e:
        print(e)
    for rows in results:
        print (list(rows))

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [11]:
session.execute("""
    CREATE TABLE IF NOT EXISTS songs_by_session (
        sessionId int,
        itemInSession int,
        artist text,
        song text,
        length float,
        PRIMARY KEY (sessionId, itemInSession))
        """)

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_by_session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        ##Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(sessionId), int(itemInSession), artist, song, float(length)))

## Select statement:
Extract these columns (artist, song title and song's length) from songs_by_session tables with these conditions (session Id = 338, and itemS Session = 4)
## Partition key: 
Columns (sessionId) and (itemInSession) must be placed to correctly define the data as some of them are shared with column (sessionId), so we have added column (itemInSession) with them.

In [13]:
##Add in the SELECT statement to verify the data was entered into the table
execute_query("""
select artist,song,length from songs_by_session 
where sessionId = 338 and itemInSession = 4
""")

['Faithless', 'Music Matters (Mark Knight Dub)', 495.30731201171875]


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
session.execute("""
        create table if not exists songs_by_user (
        userId int,
        sessionId int,
        itemInSession int,
        artist text,
        firstName text,
        lastName text,
        song text,
        PRIMARY KEY (userId, sessionId, itemInSession)
        
    )
""")

                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO songs_by_user (userId, sessionId, itemInSession, artist, firstName, lastName , song)"
        query = query + " VALUES (%s, %s, %s, %s, %s,%s,%s)"
        #artist, firstName, gender, itemInSession, lastName, length, level, location, sessionid, song, userId = line
        #session.execute(query, (int(userId), int(sessionId), int(itemInSession),   artist, firstName, lastName , song))
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4],line[9]))

## Select statement:
Extract these columns(artist, song, user first and last name) with these conditions (userid = 10, sessionid = 182) from songs_by_user table.

## Partition key:
In this case, we filter the data on these keys (userId) and (sessionId) and add the third PK (itemInSession) for ordaring.

In [16]:
execute_query(""" select artist , song , firstName , lastName from songs_by_user
where userId = 10 and sessionId = 182
""")

['Down To The Bone', "Keep On Keepin' On", 'Sylvie', 'Cruz']
['Three Drives', 'Greece 2000', 'Sylvie', 'Cruz']
['Sebastien Tellier', 'Kilometer', 'Sylvie', 'Cruz']
['Lonnie Gordon', 'Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', 'Sylvie', 'Cruz']


In [17]:
session.execute("""
create table if not exists users_by_song_plays (
song text, 
userId int, 
firstName text,
lastName text,
primary key (song, userId))
""")

                    

In [18]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song_plays (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

## Select statement:
Extract every user name (first and last) with this conditions (song = 'All Hands Against His Own') from users_by_song_plays table.

## Partition key:
We identified a column (userId) to identify users and extract the names, then we link them with the songs column and put the song (All Hands Against His Own) as a condition

In [19]:
execute_query("""
select userId, firstName, lastName from users_by_song_plays
where song='All Hands Against His Own'
""")

[29, 'Jacqueline', 'Lynch']
[80, 'Tegan', 'Levine']
[95, 'Sara', 'Johnson']


In [20]:
df = pd.read_csv('event_datafile_new.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


### Drop the tables before closing out the sessions

In [21]:
#Drop the table before closing out the sessions
session.execute("drop table songs_by_session ")
session.execute("drop table songs_by_user ")
session.execute("drop table users_by_song_plays ")

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()